In [6]:
!python train.py --network mobile0.25 --training_dataset ../0.dataset/face_detection/train/label.txt

Traceback (most recent call last):
  File "train.py", line 14, in <module>
    from models.retinaface import RetinaFace
  File "d:\python\object_detection\Pytorch_RetinaFace\models\retinaface.py", line 3, in <module>
    import torchvision.models.detection.backbone_utils as backbone_utils
  File "c:\Users\ForYou\anaconda3\envs\pytorch\lib\site-packages\torchvision\__init__.py", line 7, in <module>
    from torchvision import datasets
  File "c:\Users\ForYou\anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\__init__.py", line 1, in <module>
    from .lsun import LSUN, LSUNClass
  File "c:\Users\ForYou\anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\lsun.py", line 2, in <module>
    from PIL import Image
  File "c:\Users\ForYou\anaconda3\envs\pytorch\lib\site-packages\PIL\Image.py", line 100, in <module>
    from . import _imaging as core
ImportError: DLL load failed while importing _imaging: 지정된 모듈을 찾을 수 없습니다.


In [9]:
import torch
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

# 1. Training

In [11]:
# Retina Face train.py 폴더
cd G:/내 드라이브/Pytorch_Retinaface

G:\내 드라이브\Pytorch_Retinaface


In [21]:
!python train.py --network mobile0.25 --training_dataset ../dataset/wider_face/train/label.txt

^C
